>머신러닝과 데이터분석 A-Z 올인원 패키지 Online :『고객데이터 분석 강좌』 를 참조한 notebook

# <데이터 정보>
This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

- Source : Dr Daqing Chen, Director: Public Analytics group. chend '@' lsbu.ac.uk, School of Engineering, London South Bank University, London SE1 0AA, UK.(원문) : https://archive.ics.uci.edu/ml/datasets/Online+Retail#


# <데이터 명세>

1. InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
2. StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
3. Description: Product (item) name. Nominal.
4. Quantity: The quantities of each product (item) per transaction. Numeric.
5. InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
6. UnitPrice: Unit price. Numeric, Product price per unit in sterling.
7. CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
8. Country: Country name. Nominal, the name of the country where each customer resides.

-----------------------------

# 0. 기본 설정

In [2]:
import pandas as pd
from pandas import Series
import seaborn as sns
from pandas import DataFrame

import matplotlib.pyplot as plt

In [3]:
import matplotlib
from matplotlib import font_manager, rc
import platform

try : 
    if platform.system() == 'Windows':
    # 윈도우인 경우
        font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
        rc('font', family=font_name)
    else:    
    # Mac 인 경우
        rc('font', family='AppleGothic')
except : 
    pass
matplotlib.rcParams['axes.unicode_minus'] = False   

# 1. 데이터불러오기

In [8]:
retail = pd.read_excel('./data/OnlineRetail.xlsx')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


#### 컬럼 확인하기
 - columns 속성으로 확인
 - 컬럼
  - invoiceNo: 주문 번호
  - StockCode: 아이템 아이디
  - Description: 상품 설명
  - Quantity: 상품 주문 수량
  - InvoiceDate: 주문 시각
  - UnitPrice: 상품 가격(동일한 통화)
  - CustomerID: 고객 아이디
  - Country: 고객 거주 지역(국가)

In [ ]:
retail.columns

#### 데이터 살펴보기
 1. 데이터 분석의 가장 첫 단계
 2. 데이터를 대략적으로 파악 가능(타입, 저장된 형태)
 3. 데이터 cleansing 전략 수립

In [ ]:
retail.head()

In [ ]:
retail.info()

In [ ]:
retail.describe()

#### Data cleansing
 - null 데이터 처리
  - CustomerID 
 - Business 로직에 맞지 않은 데이터 처리
  - 음수의 아이템 수량
  - 가격이 0원 

In [ ]:
retail.isnull().sum()

#### null customerID 제거

In [ ]:
retail = retail[pd.notnull(retail['CustomerID'])]
len(retail)

#### 비지니스 로직에 맞지 않은 데이터 제거
 - 수량, 가격 > 0

In [ ]:
retail = retail[retail['Quantity'] > 0]
retail = retail[retail['UnitPrice'] > 0]

len(retail)

In [ ]:
retail.info()

In [ ]:
retail.describe()

#### 데이터 타입 변경
 - 메모리 효율화
 - 올바른 데이터 타입 매칭

In [ ]:
retail['CustomerID'] = retail['CustomerID'].astype(np.int32)
retail.info()

#### 새로운 컬럼 추가
 - Quantity * UnitPrice는 고객의 총 지출 비용(CheckoutPrice)

In [ ]:
retail['CheckoutPrice'] = retail['UnitPrice'] * retail['Quantity']
retail.head()

#### 정제 데이터 저장

In [ ]:
retail.to_csv('./OnlineRetailClean.csv')

#### 미니 프로젝트를 통한 데이터 분석의 목표
 1. 매출 분석
 2. 고객 분석
  - 우수고객 선별
  - 고객 리텐션 분석
 3. push notification 실행 의사 결정 하기

In [ ]:
### 학습목표
1. 가상 쇼핑몰 고객 주문 데이터 파악하기
 - 현재 상황(데이터로 부터) 파악
 - 모델 수립 혹은 목표 설정

#### 데이터 셋 
 - 온라인 리테일 사이트의 2010/12 - 2011/12간의 주문 기록 데이터
 - 약 500,000건의 데이터 
 - 데이터 출처: [UCI ML Repository](https://archive.ics.uci.edu/ml/datasets/Online+Retail#)

import numpy as np
import pandas as pd

retail = pd.read_csv('./OnlineRetail.csv')

#### 컬럼 확인하기
 - columns 속성으로 확인
 - 컬럼
  - invoiceNo: 주문 번호
  - StockCode: 아이템 아이디
  - Description: 상품 설명
  - Quantity: 상품 주문 수량
  - InvoiceDate: 주문 시각
  - UnitPrice: 상품 가격(동일한 통화)
  - CustomerID: 고객 아이디
  - Country: 고객 거주 지역(국가)

retail.columns

#### 데이터 살펴보기
 1. 데이터 분석의 가장 첫 단계
 2. 데이터를 대략적으로 파악 가능(타입, 저장된 형태)
 3. 데이터 cleansing 전략 수립

retail.head()

retail.info()

retail.describe()

#### Data cleansing
 - null 데이터 처리
  - CustomerID 
 - Business 로직에 맞지 않은 데이터 처리
  - 음수의 아이템 수량
  - 가격이 0원 

retail.isnull().sum()

#### null customerID 제거

retail = retail[pd.notnull(retail['CustomerID'])]
len(retail)

#### 비지니스 로직에 맞지 않은 데이터 제거
 - 수량, 가격 > 0

retail = retail[retail['Quantity'] > 0]
retail = retail[retail['UnitPrice'] > 0]

len(retail)

retail.info()

retail.describe()

#### 데이터 타입 변경
 - 메모리 효율화
 - 올바른 데이터 타입 매칭

retail['CustomerID'] = retail['CustomerID'].astype(np.int32)
retail.info()



#### 새로운 컬럼 추가
 - Quantity * UnitPrice는 고객의 총 지출 비용(CheckoutPrice)

retail['CheckoutPrice'] = retail['UnitPrice'] * retail['Quantity']
retail.head()

#### 정제 데이터 저장

retail.to_csv('./OnlineRetailClean.csv')

#### 미니 프로젝트를 통한 데이터 분석의 목표
 1. 매출 분석
 2. 고객 분석
  - 우수고객 선별
  - 고객 리텐션 분석
 3. push notification 실행 의사 결정 하기

